In [ ]:
from bs4 import BeautifulSoup
import requests
import csv

URL = 'http://ru-pets.ru/index.php?m=6&c=2&to=1'

result = []

try:
  resp = requests.get(URL)
  soup = BeautifulSoup(resp.text, 'lxml')

  max_page = 1
  pagination_elements = soup.find_all('div', id="page")

  if pagination_elements:
    max_page = int(pagination_elements[-1].get_text().strip())

  for page in range(1, max_page+1):
    resp = requests.get(URL, params={'page': page})

    cards = soup.find_all('div', class_='listitem')
    for card in cards:

      #ищем на карточке дату - она в тэге h2
      #ищем тэг h2
      date = card.find('h2')
      #если шали такой тэг - берем из него текстовую часть - get_text()
      #обрезаем то, что после запятой - split(',')
      #берем из обрезанного первый элемент, то есть дату - [0]
      if date:
        date = date.get_text().split(',')[0]
      else:
        date = ''

      name = card.find('span', class_="cl-green")
      if name:
        name = name.get_text().strip()
      else:
        name = ''

      club = card.find('span', class_="fwb cl-804000").find_next('span', class_="fwb cl-804000")
      if club:
        club = club.get_text().strip()
      else:
        club = ''

      result.append({'date': date, 'name': name, 'club': club})


  with open('Cat_Show.csv', 'w', encoding='utf-8') as f:
    dict_writer = csv.DictWriter(f, result[0].keys())
    dict_writer.writeheader()
    dict_writer.writerows(result)


except Exception as ex:
  print(ex)